<p style="font-size: 20px; color:#24ff5e; text-align: center"> Importando Libs </p>

In [79]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from scipy.stats import pearsonr 
from IPython.display import display
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

<p style="font-size: 20px; color:#24ff5e; text-align: center"> Importando Dados </p>

In [80]:
arquivo = r'..\data\Electric_Vehicle_Population_Data.csv'

dados = pd.read_csv(arquivo, 
    dtype={
        'Postal Code': 'str',
        'VIN (1-10)': 'str',
        'DOL Vehicle ID': 'str'
    }
)

<p style="font-size: 20px; color:#24ff5e; text-align: center"> Tratando Dados </p>

In [112]:
dados['City'] = dados['City'].fillna('Desconhecida')
dados['County'] = dados['County'].fillna('Desconhecido')
dados.dropna(subset=['Base MSRP', 'Electric Range'])

if 'Vehicle Location' in dados.columns:
    dados[['Longitude', 'Latitude']] = dados['Vehicle Location'].str.extract(r'POINT \(([-\d\.]+) ([-\d\.]+)\)')

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAIS CIDADE POSSUEM A MAIOR QUANTIDADE DE VEÍCULOS ELÉTRICOS E HÍBRIDOS REGISTRADOS?
</p>

In [82]:
top_cidades = dados['City'].value_counts().head(10).reset_index(name='Vehicle Count')

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))
ax = sns.barplot(
    x='Vehicle Count',
    y='City',
    data=top_cidades,
    palette='viridis',  
    edgecolor=None,   
    linewidth=0.0,   
)

for patch in ax.patches : patch.set_height(0.6)  

ax.set_facecolor("#101010")
plt.gcf().set_facecolor("#101010")

for i, valor in enumerate(top_cidades['Vehicle Count']):
    ax.text(
        valor + 3,   
        i,          
        f"{valor:,}",  
        color="white",
        va='center', 
        fontsize=12
    )

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_color('white')
ax.xaxis.set_visible(False)
ax.grid(False)

plt.title(
    'Top 10 Cidades com Maior Quantidade de Veículos Elétricos e Híbridos Registrados', 
    fontsize=16, 
    color="white", 
    pad=40
)

ax.tick_params(colors="white")  

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAL É A DISTRIBUIÇÃO DO ALCANCE ELÉTRICO DOS VEÍCULOS REGISTRADOS?
</p>

In [84]:
frequencia_bins = pd.cut(dados['Electric Range'], bins=20).value_counts().sort_index().reset_index(name='Qtd')

In [ ]:
sns.set(style="whitegrid")

max_range = dados['Electric Range'].max()
bins = list(range(0, int(max_range) + 25, 25))  

plt.figure(figsize=(12, 6))
ax = sns.histplot(
    data=dados,
    x='Electric Range',
    bins=bins,  
    kde=True,   
    color='dodgerblue'
)

ax.set_facecolor("#101010")
plt.gcf().set_facecolor("#101010")

plt.title('Distribuição do Alcance Elétrico dos Veículos Registrados', fontsize=16, color="white", pad=20)
plt.xlabel('Alcance Elétrico (milhas)', fontsize=12, color="white")
plt.ylabel('', fontsize=12, color="white")  
plt.xticks(bins, labels=[f"{b:,}".replace(",", ".") for b in bins], color="white") 
plt.xlim(0, max_range)

ax.tick_params(axis='y', left=False, labelleft=False)  
ax.tick_params(colors="white", axis='x')  
ax.grid(False)

for patch in ax.patches : patch.set_edgecolor('none')

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)  
ax.spines['bottom'].set_color('white')

for patch in ax.patches:
    height = patch.get_height()
    if height > 0:  
        ax.text(
            patch.get_x() + patch.get_width() / 2,  
            height + 0.1,  
            f"{int(height):,}".replace(",", "."),  
            ha="center",
            va="bottom",  
            color="white",
            fontsize=10,
        )

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    COMO A ELEGIBILIDADE PARA COMBUSTÍVEL ALTERNATIVO LIMPO (CAFV Eligibility) ESTÁ DISTRIBUÍDA ENTRE OS TIPOS DE VEÍCULOS?
</p>

In [86]:
agrupados = dados.groupby(['Electric Vehicle Type', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility']).size().reset_index(name='Count')
agrupados = agrupados[agrupados['Count'] > 0]

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))
ax = sns.barplot(
    data=agrupados,
    x='Electric Vehicle Type',
    y='Count',
    hue='Clean Alternative Fuel Vehicle (CAFV) Eligibility',
    palette='viridis'  
)

ax.set_facecolor("#101010")
plt.gcf().set_facecolor("#101010")

plt.title('Distribuição da Elegibilidade para Combustível Alternativo Limpo por Tipo de Veículo',
          fontsize=16, color="white", pad=20)
plt.xlabel('Tipo de Veículo', fontsize=12, color="white")
plt.ylabel('Frequência', fontsize=12, color="white")

ax.tick_params(colors="white")
ax.set_yticks(range(0, agrupados['Count'].max() + 1, 25000))
ax.set_yticklabels([f"{int(tick):,}".replace(",", ".") for tick in ax.get_yticks()], color="white")
ax.grid(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')

legend = ax.legend(title="Elegibilidade CAFV", loc='upper right', fontsize=10, title_fontsize=12)
plt.setp(legend.get_texts(), color="black")  
plt.setp(legend.get_title(), color="black")

for patch in ax.patches : patch.set_edgecolor('none')  

for patch in ax.patches:
    height = patch.get_height()
    if height > 0:  
        ax.text(
            patch.get_x() + patch.get_width() / 2,  
            height + 100,  
            f"{int(height):,}".replace(",", "."),  
            ha="center",
            va="bottom",
            color="white",
            fontsize=10,
        )

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAL É A CORRELAÇÃO ENTRE O PREÇO BASE (Base MSRP) E O ALCANCE ELÉTRICO DOS VEÍCULOS?
</p>

In [88]:
dados_limpos = dados.dropna(subset=['Base MSRP', 'Electric Range'])
correlacao, _ = pearsonr(dados_limpos['Base MSRP'], dados_limpos['Electric Range'])

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.regplot(
    data=dados_limpos,
    x='Base MSRP',
    y='Electric Range',
    scatter_kws={'color': 'dodgerblue', 'alpha': 0.6},  
    line_kws={'color': 'lime', 'alpha': 0.8},  
)

plt.title(f'Correlação entre Preço Base e Alcance Elétrico\nCoeficiente de Correlação: {correlacao:.2f}',
          fontsize=16, color="white", pad=20)
plt.xlabel('Preço Base (USD)', fontsize=12, color="white")
plt.ylabel('Alcance Elétrico (milhas)', fontsize=12, color="white")

ax.tick_params(colors="white")
ax.set_facecolor("#101010")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')

plt.gcf().set_facecolor("#101010")

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAIS SÃO AS ÁREAS COM MAIOR CONCENTRAÇÃO DE VEÍCULOS ELEGÍVEIS PARA CAFV?
</p>

In [90]:
elegiveis_por_area = dados[dados['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible'] \
    .groupby('City').size().reset_index(name='Count')


elegiveis_por_area = elegiveis_por_area.sort_values(by='Count', ascending=False)
dados_filtrados = dados.dropna(subset=['City', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility'])
elegiveis_por_area = dados_filtrados[dados_filtrados['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible']

agrupados_por_cidade = elegiveis_por_area.groupby('City').size().reset_index(name='Count')
agrupados_por_cidade = agrupados_por_cidade[agrupados_por_cidade['Count'] > 0]
agrupados_por_cidade = agrupados_por_cidade.sort_values(by='Count', ascending=False).head(10)

categorias = agrupados_por_cidade['City']
valores = agrupados_por_cidade['Count']

max_valor = max(valores)
valores_normalizados = [v / max_valor for v in valores]

In [ ]:
angulos = np.linspace(0, 2 * np.pi, len(categorias), endpoint=False).tolist()

valores_normalizados += valores_normalizados[:1]
angulos += angulos[:1]

fig, ax = plt.subplots(figsize=(8, 8), dpi=100, subplot_kw=dict(polar=True))

ax.set_ylim(0, 1) 
ax.fill(angulos, valores_normalizados, color='dodgerblue', alpha=0.6)
ax.plot(angulos, valores_normalizados, color='dodgerblue', linewidth=2)

ax.set_yticklabels([]) 
ax.set_xticks(angulos[:-1])
ax.set_xticklabels(categorias, color='white', fontsize=12)

ax.set_title('Concentração de Veículos Elegíveis para CAFV por Cidade', fontsize=16, color="white", pad=40)
ax.set_facecolor("#101010")
fig.patch.set_facecolor('#101010')

ax.spines['polar'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
elegiveis_por_area = dados[dados['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible'] \
    .groupby('City').size().reset_index(name='Count')

elegiveis_por_area = elegiveis_por_area.sort_values(by='Count', ascending=False)
dados_filtrados = dados.dropna(subset=['City', 'Clean Alternative Fuel Vehicle (CAFV) Eligibility'])
elegiveis_por_area = dados_filtrados[dados_filtrados['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] == 'Clean Alternative Fuel Vehicle Eligible']

agrupados_por_cidade = elegiveis_por_area.groupby('City').size().reset_index(name='Count')
agrupados_por_cidade = agrupados_por_cidade[agrupados_por_cidade['Count'] > 0]
agrupados_por_cidade = agrupados_por_cidade.sort_values(by='Count', ascending=False).head(10)

categorias = agrupados_por_cidade['City']
valores = agrupados_por_cidade['Count']

max_valor = max(valores)
valores_normalizados = [v / max_valor for v in valores]

angulos = np.linspace(0, 2 * np.pi, len(categorias), endpoint=False).tolist()

fig = plt.figure(figsize=(8, 8), dpi=100)
ax = fig.add_subplot(projection='polar')

c = ax.scatter(angulos, valores_normalizados, c=valores_normalizados, s=200, cmap='hsv', alpha=0.75)

ax.set_ylim(0, 1)
ax.set_yticklabels([]) 
ax.set_xticks(angulos)
ax.set_xticklabels(categorias, color='white', fontsize=12)

ax.set_title('Concentração de Veículos Elegíveis para CAFV por Cidade', fontsize=16, color="white", pad=40)
ax.set_facecolor("#101010")
fig.patch.set_facecolor('#101010')

ax.spines['polar'].set_visible(False)

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAIS MODELOS (Model) POSSUEM A MAIOR PARTICIPAÇÃO NA BASE DE VEÍCULOS?
</p>

In [92]:
modelos_populares = dados['Model'].value_counts().reset_index()
modelos_populares.columns = ['Model', 'Count']

top_modelos = modelos_populares.head(10)
top_modelos = top_modelos.merge(dados[['Model', 'Make']].drop_duplicates(), on='Model', how='left')

top_modelos['Percentage'] = (top_modelos['Count'] / top_modelos['Count'].sum()) * 100
top_modelos['Label'] = top_modelos['Make'] + " - " + top_modelos['Model']

labels = top_modelos['Label']
sizes = top_modelos['Percentage']
colors = sns.color_palette('viridis', len(sizes))

In [ ]:
plt.figure(figsize=(9, 9))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=None,  
    autopct='%1.1f%%',
    startangle=90,
    colors=colors,
    wedgeprops={'edgecolor': 'black', 'linewidth': 1},  
    pctdistance=0.60,  
    textprops={'fontsize': 10, 'color': 'white'} 
)

centre_circle = plt.Circle((0, 0), 0.70, fc='#101010')
plt.gca().add_artist(centre_circle)

for i, wedge in enumerate(wedges):
    angle = (wedge.theta2 - wedge.theta1) / 2 + wedge.theta1
    x = 1 * np.cos(np.radians(angle))  
    y = 1 * np.sin(np.radians(angle))
    label_x = 1.2 * np.cos(np.radians(angle))  
    label_y = 1.2 * np.sin(np.radians(angle))

    plt.plot([x, label_x], [y, label_y], color='white', lw=0.8)  
    plt.text(
        label_x, label_y, labels[i],
        ha=('left' if label_x > 0 else 'right'), 
        va='center', fontsize=10, color='white'
    )

plt.title('Participação Percentual dos Top 10 Modelos por Fabricante', fontsize=16, color="white", pad=40)
plt.gcf().set_facecolor("#101010")
plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    EXISTE UMA RELAÇÃO ENTRE O DISTRITO LEGISLATIVO (Legislative District) E A QUANTIDADE DE VEÍCULOS ELÉTRICOS REGISTRADOS?
</p>

In [94]:
contagem_distritos = dados['Legislative District'].value_counts().reset_index()
contagem_distritos.columns = ['Legislative District', 'Quantidade de Veículos']

bins = [0, 1000, 5000, 10000, 20000, 30000]  
labels = ['0-1k', '1k-5k', '5k-10k', '10k-20k', '20k-30k'] 
contagem_distritos['Faixa de Veículos'] = pd.cut(contagem_distritos['Quantidade de Veículos'], bins=bins, labels=labels)

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(12, 8))
ax = sns.scatterplot(
    data=contagem_distritos,  
    x='Legislative District',  
    y='Quantidade de Veículos', 
    hue='Faixa de Veículos',  
    palette='plasma',  
    size='Quantidade de Veículos', 
    sizes=(100, 1000), 
    marker='o',  
    edgecolor=None, 
    alpha=0.7,  
    legend=None  
)

ax.set_facecolor("#101010")  
plt.gcf().set_facecolor("#101010")  
ax.grid(True, linestyle='--', color='white', alpha=0.1)  

plt.title('Relação entre o Distrito Legislativo e a Quantidade de Veículos Elétricos',
          fontsize=18, color="white", pad=20)
plt.xlabel('Distrito Legislativo', fontsize=14, color="white")
plt.ylabel('Quantidade de Veículos', fontsize=14, color="white")


ax.set_xticks(range(0, len(contagem_distritos), int(len(contagem_distritos) / 10)))  # Ajuste para 10 ticks
ax.set_xticklabels(
    contagem_distritos['Legislative District'].iloc[::int(len(contagem_distritos) / 10)],  # Ajuste de rótulos
    rotation=90, 
    fontsize=12, 
    color="white"
)

ax.tick_params(colors="white")
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=12, color="white")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')

for collection in ax.collections:
    collection.set_edgecolor('none')  
    collection.set_linewidth(0.5)  

plt.subplots_adjust(right=0.9, left=0.1, top=0.9, bottom=0.1)
plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    COM BASE NA TAXA DE CRESCIMENTO ANUAL, QUAL SERIA A QUANTIDADE PROJETADA DE VEÍCULOS ELÉTRICOS PARA 2025?
</p>

In [96]:
dados_2024 = dados[dados['Model Year'] == 2024]
quantidade_ano = dados.groupby('Model Year').size()
taxa_crescimento = (quantidade_ano[2024] - quantidade_ano[2023]) / quantidade_ano[2023]
quantidade_2025 = round(quantidade_ano[2024] * (1 + taxa_crescimento), 2)

In [ ]:
anos = [2023, 2024, 2025]
quantidades = [quantidade_ano[2023], quantidade_ano[2024], quantidade_2025]

percentual_2024 = (quantidade_ano[2024] - quantidade_ano[2023]) / quantidade_ano[2023] * 100
percentual_2025 = (quantidade_2025 - quantidade_ano[2024]) / quantidade_ano[2024] * 100

plt.figure(figsize=(12, 8))
plt.plot(anos, quantidades, marker='o', color='red', linestyle='-', markersize=8, label='Projeção de Veículos Elétricos')
plt.fill_between(anos, quantidades, color='lightcoral', alpha=0.5)

plt.title('Projeção de Veículos Elétricos até 2025', fontsize=16, color="white", pad=20, weight='bold')
plt.xlabel('Ano', fontsize=12, color="white")
plt.ylabel('Quantidade de Veículos', fontsize=12, color="white")

plt.xticks(anos, color="white")
plt.yticks(color="white")

plt.gcf().set_facecolor("#101010")
plt.gca().set_facecolor("#101010")

plt.grid(False)

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_color('white')
plt.gca().spines['bottom'].set_color('white')

for i, (x, y) in enumerate(zip(anos, quantidades)):
    percentual = percentual_2024 if x == 2024 else percentual_2025 if x == 2025 else 0
    rotulo = f"{int(y):,}".replace(",", ".") + f"\n({percentual:+.2f}%)"
    plt.text(x, y + 2000, rotulo, ha="center", va="bottom", color="white", fontsize=12, weight='bold')

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left">
    QUAL É A PROPORÇÃO DE VEÍCULOS POR TIPO DE COMBUSTÍVEL (BEV vs. PHEV) EM CADA ANO?
</p>

In [ ]:
proporcao_ano_tipo = dados.groupby(['Model Year', 'Electric Vehicle Type']).size().unstack(fill_value=0)

proporcao_ano_tipo['Proporcao BEV'] = proporcao_ano_tipo['Battery Electric Vehicle (BEV)'] / proporcao_ano_tipo.sum(axis=1)
proporcao_ano_tipo['Proporcao PHEV'] = proporcao_ano_tipo['Plug-in Hybrid Electric Vehicle (PHEV)'] / proporcao_ano_tipo.sum(axis=1)

proporcao_ano_tipo[['Proporcao BEV', 'Proporcao PHEV']]

In [ ]:
ax = proporcao_ano_tipo[['Proporcao BEV', 'Proporcao PHEV']].plot(kind='bar', stacked=True, figsize=(12, 8), color=['lightblue', 'lightcoral'])

ax.set_title('Proporção de Veículos Elétricos por Tipo de Combustível (BEV vs PHEV)', fontsize=16, color="white", pad=20, weight='bold')
ax.set_xlabel('Ano', fontsize=12, color="white")
ax.set_ylabel('Proporção', fontsize=12, color="white")

ax.tick_params(axis='x', colors="white")
ax.tick_params(axis='y', colors="white")

plt.gcf().set_facecolor("#101010")
plt.gca().set_facecolor("#101010")

ax.grid(False)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')

plt.tight_layout()
plt.show()

<p style="font-size: 20px; color:#00d0ff; text-align: left"> 
    QUAL É A DISTRIBUIÇÃO DA QUANTIDADE DE VEÍCULOS POR FAIXA DE ALCANCE ELÉTRICO (e.g., 0-50, 51-100, 101-200, etc.)?
</p>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

sns.kdeplot(
    data=dados,
    x='Electric Range',  
    shade=True, 
    color='skyblue', 
    alpha=0.5,  
    linewidth=3 
)

sns.scatterplot(
    x=dados['Electric Range'], 
    y=[0]*len(dados),  
    color='red',  
    alpha=0.7,  
    edgecolor=None, 
    s=200  
)

plt.title('Distribuição de Veículos Elétricos por Alcance Elétrico', fontsize=16, color="white", pad=20, weight='bold')
plt.xlabel('Alcance Elétrico (km)', fontsize=12, color="white")
plt.ylabel('Densidade', fontsize=12, color="white")

plt.gcf().set_facecolor("#101010")
plt.gca().set_facecolor("#101010")

plt.xticks(color="white")
plt.yticks(color="white")

plt.grid(False)

plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_color('white')
plt.gca().spines['bottom'].set_color('white')

plt.tight_layout()
plt.show()